In [1]:
import os
import sys
import multiprocessing
import warnings

import pandas as pd
import numpy as np

from Bio import SeqIO
from glob import glob
from functools import partial
sys.path.append('../../')
from hamp_pred.external.lbs.sequence import mmseqs2

In [2]:
data_dir = '../../data/input'
out_path = os.path.join(data_dir, 'af2_full')

# Get HAMP sequences

In [3]:
# read and cluster sequences from https://pubmed.ncbi.nlm.nih.gov/20184894/
msa = list(SeqIO.parse(os.path.join(data_dir, 'hamp_msa.fasta'), 'fasta'))

msa_df = pd.DataFrame(
        [(str(i.seq).replace('-', ''), i.id) for i in msa],
    columns=['sequence', 'id']
)

In [4]:
len(msa)

6456

In [5]:
# remove redundancy at 70% ident and 70% coverage
cl = mmseqs2.MMSeqsClusterer()
msa_df_clustered = cl.cluster(msa_df, min_identity=0.7, coverage=0.7)

createdb /tmp/s_7r99v0 tmp/2277846130962167930/input --max-seq-len 65535 --dont-split-seq-by-len 1 --dbtype 0 --dont-shuffle 1 --id-offset 0 --compressed 0 -v 3 

Converting sequences
[
Time for merging files: 0h 0m 0s 8ms
Time for merging files: 0h 0m 0s 6ms
Time for merging files: 0h 0m 0s 0ms
Time for processing: 0h 0m 0s 42ms
kmermatcher tmp/2277846130962167930/input tmp/2277846130962167930/clu_tmp/9752797917747901279/linclust/13067973857597857346/pref --sub-mat blosum62.out --alph-size 13 --min-seq-id 0.7 --kmer-per-seq 21 --adjust-kmer-len 0 --mask 0 --mask-lower-case 0 --cov-mode 0 -k 0 -c 0.7 --max-seq-len 65535 --hash-shift 5 --split-memory-limit 0 --include-only-extendable 0 --skip-n-repeat-kmer 0 --threads 20 --compressed 0 -v 3 

Database size: 6456 type: Aminoacid
Reduced amino acid alphabet: (A S T) (C) (D B N) (E Q Z) (F Y) (G) (H) (I V) (K R) (L J M) (P) (W) (X) 

Estimated memory consumption 2 MB
Generate k-mers list for 1 split
[=======================================

[=================================================================] 5.56K 0s 4ms
Index statistics
Entries:          83252
DB size:          488 MB
Avg k-mer size:   0.001301
Top 10 k-mers
    DEGLTF	82
    EIQAFN	62
    DEGLNF	49
    EIDAFN	35
    RREGAR	32
    EIQSFN	21
    PVSDGQ	20
    IGMREV	19
    DEGMTV	16
    PVSDGR	14
Time for index table init: 0h 0m 0s 523ms
k-mer similarity threshold: 154
	k-mers per position = 0.475694, k-mer match probability: 0.000000
k-mer match probability: 0.000000

Starting prefiltering scores calculation (step 1 of 1)
Query db start  1 to 5561
Target db start  1 to 5561
[=================================================================] 5.56K 0s 4ms

0.473296 k-mers per position
27 DB matches per sequence
0 overflows
6 sequences passed prefiltering per query sequence
3 median result list length
0 sequences with 0 size result lists

Time for prefiltering scores calculation: 0h 0m 0s 9ms
Time for merging files: 0h 0m 0s 7ms
Time for processing: 0h 0m 0s

In [6]:
msa_df_clustered = msa_df_clustered.groupby('clust_id').head(1)
len(msa_df_clustered)

5388

In [7]:
msa_df_clustered.to_pickle(os.path.join(data_dir, 'hamp_master.p'))

# Unpack AF2 models

In [8]:
def extract_models(models_dir, out_path, df_clustered):
    for out_file in glob(os.path.join(models_dir, '*', '*result.zip')):

        #print(out_file)
        
        seq_id = int(out_file.split('/')[-1].replace('.result.zip', '').split("_")[1])
        assert seq_id in df_clustered.index

        tmp = os.path.join(out_path, str(seq_id))
        
        if not os.path.exists(tmp):
        
            os.system(f'mkdir -p {tmp}')

            # -n ensures that existing files are not overwritten
            os.system(f'unzip -n -j "{out_file}" "*relaxed*" -d {tmp}')
        
        #break

In [9]:
#
# af2 model files are avaliable upon request 
#
df_clustered = pd.read_pickle(os.path.join(data_dir, 'hamp_master.p'))

# get aligned sequences
msa = list(SeqIO.parse(os.path.join(data_dir, 'hamp_msa.fasta'), 'fasta'))
hampid2alnseq = pd.DataFrame(
        [(str(i.seq), i.id) for i in msa],
    columns=['sequence', 'id']
)

In [10]:
len(df_clustered), len(msa), len(hampid2alnseq)

(5388, 6456, 6456)

In [11]:
# unzip af2 models bundles
# uncomment me
models_dir = '/home/nfs/jludwiczak/calc/hamp_olek/hamp_final/out'

extract_models(models_dir, out_path, df_clustered)

# Analyses AF2 models


In [12]:
### Measure with SamCC
### Get AF2 scores
### Store PDB file link

In [13]:
# To measure af2 hamp models use samCC turbo https://academic.oup.com/bioinformatics/article/36/22-23/5368/6039120
sys.path.append('../../hamp_pred/')
from utils.measure import measure_one_HAMP, get_ref_crick
from utils.tools import diffangle

In [14]:
def run_multiprocess(func, tasks, n_cores, tasks_per_core=1):  
    stdout_queue = multiprocessing.Queue()
    pool = multiprocessing.Pool(processes=n_cores, initargs=[stdout_queue], maxtasksperchild=tasks_per_core)
    for i, data in enumerate(pool.map(func, tasks), 1):
        yield data
    pool.close()
    pool.join()

In [15]:
# referece Crick angles
crangles = {'a':19.5,'b':122.35,'c':-134.78,'d': -31.92,'e':70.92 ,'f':173.78,'g':-83.35}

# aa names mapping
AA_3_to_1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
             'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
             'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
             'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

In [16]:
# define ranges for helix 1 and helix 2 in MSA
h1_msa_start = 5 #4
h1_msa_stop = 18

h2_msa_start = 85 # 84
h2_msa_stop = 98

start_hep = 'a' #g

In [17]:
warnings.filterwarnings(action='ignore', category=UserWarning)
import json

In [19]:

data=[]
tmp_idx=-1
for idx in df_clustered.index:
    
    if tmp_idx >= 5000:
        af2_path = '/home/nfs/sdunin/tmp/hamp'
    else:
        af2_path = '/home/nfs/rmadaj/hamp/HAMPpred/clustering/af2_structures'

    
    row = df_clustered.loc[idx]
    group = row['id'].split("|")[1]
    true_id = int(row['id'].split("|")[0])
        
    # aligned sequence
    alnseq = hampid2alnseq.loc[idx].sequence
    
    # cut helix1 and helix2
    h1 = alnseq[h1_msa_start:h1_msa_stop].replace('-', '')
    h2 = alnseq[h2_msa_start:h2_msa_stop].replace('-', '')
    
    if len(h1) != len(h2): 
        continue
        
    seq = row.sequence    
    if seq.find('X')>-1: 
        continue

    # measure model        
    h1_start = seq.find(h1); assert h1_start > -1
    h2_start = seq.find(h2); assert h2_start > -1
    
    a1_start, a1_stop = h1_start+1, h1_start+len(h1)+1
    a2_start, a2_stop = h2_start+1, h2_start+len(h2)+1
    chain1, chain2 = 'A', 'B'
    
    kwargs = {'a1_start':a1_start, 
              'a1_stop':a1_stop,
              'a2_start':a2_start,
              'a2_stop':a2_stop,
              'chain1':chain1,
              'chain2':chain2}
    
    
    
    mapfunc = partial(measure_one_HAMP, **kwargs)
    
    # get 1 rank af2 model
    # old run
    # hamp_315_A_group_44_unrelaxed_rank_001_alphafold2_multimer_v3_model_5_seed_000.pdb

    #pdb_files = sorted(glob(os.path.join(af2_path, str(idx), '*_relaxed*.pdb')), key=lambda x:int(x.split('/')[-1].split("_")[4]))
    
    tmp_idx +=1 
    
    #tmp_idx = df_clustered.index.get_loc(idx)
    #print(idx, tmp_idx)
    
    pdb_files = glob(os.path.join(af2_path, str(tmp_idx), '*_unrelaxed*rank_001*.pdb'))
    
    # analyse only rank 1 model
    pdb_files = [pdb_files[0]]
        
    # analyse selected models 
    for job, pdb_file in zip(run_multiprocess(mapfunc, pdb_files, len(pdb_files)), pdb_files):    
                
        dir_path, filename = os.path.split(pdb_file)
        _, last_dir = os.path.split(dir_path)
        assert tmp_idx == int(last_dir)
        pdb_file_simple = os.path.join(last_dir, filename)
        
        # parse scores 'max_pae', 'pae', 'plddt', 'ptm'
        json_file = pdb_file[:-4].replace('_unrelaxed_', '_scores_') + ".json"
        scores = json.load(open(json_file))
        
        # measure with SamCC
        
        bundle_df, n_crick_mut, c_crick_mut = job
        
        #    crick = bundle_df.crick.values # in measure_one_HAMP
        #    n_crick = crick[0::2]
        #    c_crick = crick[1::2]
        
        
        n_shift = bundle_df['shift'][0::2].mean()
        c_shift = bundle_df['shift'][1::2].mean()
        
        n_radius = bundle_df['radius'][0::2].mean()
        c_radius = bundle_df['radius'][1::2].mean()
        
        n_A = bundle_df['A'][0::2].mean()
        c_A = bundle_df['A'][1::2].mean()
        
        nn_P = bundle_df['P'][0::2].mean()
        cc_P = bundle_df['P'][1::2].mean()
        
        n_crick_mut = n_crick_mut[2:-2]
        c_crick_mut = c_crick_mut[2:-2]
        
        # assume canonical bundle periodicity for calculating reference Crick angles
        n_P = c_P = 3.5 

        c_phi = n_phi = crangles[start_hep] 

        n_crick_ref = get_ref_crick(n_P, n_phi)[:len(n_crick_mut)]
        c_crick_ref = get_ref_crick(c_P, c_phi)[:len(c_crick_mut)]

        n_crick_diff = diffangle(n_crick_mut, n_crick_ref)
        n_crick_diff = (n_crick_diff[0::2] + n_crick_diff[1::2])/2

        c_crick_diff = diffangle(c_crick_mut, c_crick_ref)
        c_crick_diff = (c_crick_diff[0::2] + c_crick_diff[1::2])/2

        n_crick_diff = np.mean(n_crick_diff)
        c_crick_diff = np.mean(c_crick_diff)

        # rotation asymmetry
        crick_diff = diffangle(n_crick_diff, c_crick_diff) 

        seq1 = bundle_df.res_name[0::4].tolist()
        seq2 = bundle_df.res_name[1::4].tolist()
        seq1 = "".join([AA_3_to_1[res] for res in seq1])
        seq2 = "".join([AA_3_to_1[res] for res in seq2])
        
        assert seq1 == h1 and seq2 == h2

        # add record
        
        print(h1, h2, idx, tmp_idx, group)
        
        data.append(
                (true_id, group, n_crick_diff, c_crick_diff, crick_diff, \
                 h1, h2, n_crick_mut, c_crick_mut, seq, pdb_file_simple,
                 scores['max_pae'], np.mean(scores['plddt']), scores['ptm'], np.mean(scores['pae']),
                 n_shift, c_shift, n_radius, c_radius, n_A, c_A, nn_P, cc_P
                )
            )
        
    #if len(data)>10:break
        
    # debug
    #break
        
data_df = pd.DataFrame(data, columns=['idx', 'group', 'n_rot', 'c_rot', 'rot', 
                                      'n_seq', 'c_seq', 'n_crick_mut', 'c_crick_mut', 'full_seq', 'pdb_file',
                                      'max_pae', 'plddt', 'ptm', 'pae_mean', 
                                      'n_shift', 'c_shift', 'n_radius', 'c_radius', 'n_A', 'c_A', 'n_P', 'c_P'])
assert data_df['idx'].is_unique
data_df.set_index('idx', inplace=True)
len(data_df)

PLKELVQGVQRIA EIGELITSFNLMA 0 0 A_group_13
PIRELVEGVRSIA ELGELIASFNDMA 1 1 A_group_13
PIKELLDGVKNIA ELGELIVNFNEMA 2 2 A_group_13
PLSELRKGIRRVA ELGDLILQFNDMG 14 3 A_group_13
PVKELLRGVRAVA ELGELVNGFNDMA 15 4 A_group_13
PIKELLAGIRNIS QIGELIFSFNNMA 16 5 A_group_13
PQKKLLLGIQNIA QLSTLIISFNEMA 17 6 A_group_13
PLDILSNGVMQIS EFAPVCADFNEMA 18 7 A_group_14
PLELLSYGAEQIK EFGQVCGDFDEMR 19 8 A_group_14
PLELLSYGAGQIE EFAKVCRDFDKMR 20 9 A_group_14
PLQKLSRGTKEIK EFGEVCRDFDEMR 21 10 A_group_14
PLVKLKQAAEKMG EVGELCESFENMR 22 11 A_group_14
PVGKMREATKSIK ELGQLCGDLEDMR 23 12 A_group_14
PLQQLLEATQKIA EMGALCEAFDLMR 24 13 A_group_14
PIRKLSEAAGKIS ELGQLSNTFEDMR 25 14 A_group_14
PLDVLKRATKEMK EIGQLCEDFEEMR 26 15 A_group_14
PLSQLKQASEQIK EIGELFVAFEEMR 27 16 A_group_14
PLKELQYASNEIR EFGEAIKSFEGMR 28 17 A_group_14
PLVKLRKATQNIK EFSELCRDFEEMR 29 18 A_group_14
PLAKLQAAARNIK EIGQLCQDFEEMR 30 19 A_group_14
PLNKLQEATKKIR EIGQLCQDFEEMR 31 20 A_group_14
PLNQLKIGTEKIK EIGELCDAFDSMR 32 21 A_group_14
PITALKEGAQRIK EIGQLNRSF

RIRKIENIAQRVS EIGGLAAAFNRMK 198 136 A_group_26
PLKKMEQLAQRIS EMGMLAASLNRMK 200 137 A_group_26
PLKKMEQLAQRIS EMGMLAASLNRMK 200 137 A_group_26
PIKKMAQLAQKIS EVGMLAASLNRMK 201 138 A_group_26
PITNMSKWSRGVS EIGVLAASVNRLK 202 139 A_group_26
PIVRMAQTAEQVS EIGMLGKAFNRMK 203 140 A_group_26
PLQKMAKIAESVS EIGALAKAFNRMK 204 141 A_group_26
PITHLTRVAEQVS EVGKLAEAFTRMQ 205 142 A_group_26
PITHLTRVAEQVS EVGKLAEAFTRMQ 205 142 A_group_26
SIKKLSTAAYAIS ELVILAESFNKMT 208 143 A_group_27
PITQLSIAAAKIA ELEILAQSFNQMS 209 144 A_group_27
PVLRLTEASREIA ELRTLAHSFNRMT 210 145 A_group_27
PVLRLTEASREIA ELRTLAHSFNRMT 210 145 A_group_27
PLLRVARAAEAIS EIDQLARSFNRMS 211 146 A_group_27
PLLRVARAAEAIS EIDQLARSFNRMS 211 146 A_group_27
PLMSLSQASQEIA ELRILATVFNQMS 212 147 A_group_27
PISRLSQASQAIA ELRILSQSFKTMN 213 148 A_group_27
PISRLSQASQAIA ELRILSQSFKTMN 213 148 A_group_27
PILRMNLASQAIA ELNILTQSFNYMA 215 149 A_group_27
PILRMNLASQAIA ELNILTQSFNYMA 215 149 A_group_27
PILQLNRAAEAIA ELEVLAHSFNQMA 216 150 A_group_27
PILQLNRAAEAIA

PIRAVQEGASRLA EVEVLADEFNRMA 322 238 A_group_44
PIQALNSGAQRLG ELEQLAARFNSMA 323 239 A_group_44
PIQALNSGAQRLG ELEQLAARFNSMA 323 239 A_group_44
PITALRDGAHKLG ELEDLAGQFNRMA 324 240 A_group_44
PITALRDGAHKLG ELEDLAGQFNRMA 324 240 A_group_44
PLGRLSAASREIA ELGELTGSFNAMS 326 241 A_group_44
PLGRLSAASREIA ELGELTGSFNAMS 326 241 A_group_44
PIRRLEVGTQKIS EIGDLTAAFNEMS 327 242 A_group_44
PIRRLEVGTQKIS EIGDLTAAFNEMS 327 242 A_group_44
PIRRLQAGAREIS EIGELTAAFNDMS 330 243 A_group_44
PIKQLRAGACRIG EFGELAVAFNYMS 332 244 A_group_44
PIKQLRAGACRIG EFGELAVAFNYMS 332 244 A_group_44
PIQGLTRQAAAVA EIGRLSSAFNDMT 333 245 A_group_5
PIQGLTRQAAAVA EIGRLSSAFNDMT 333 245 A_group_5
PIKELTKHARAVA EIGQLSQAFNYMT 334 246 A_group_5
PISDMRRQAIEMA EIGQLALSFNNLS 335 247 A_group_5
PISDMRRQAIEMA EIGQLALSFNNLS 335 247 A_group_5
PLSDMRKQAIELA EIGQLATTFNYLT 337 248 A_group_5
PLSDMRKQAIELA EIGQLATTFNYLT 337 248 A_group_5
PIEEMKRQTARIA ELGQLAQAVNNLS 338 249 A_group_5
PIGEMREQAIRIA ELGQLADTFNQLA 339 250 A_group_5
PIVEMRRQADFLG EIGQLSAT

PLSELAAGTRAVA ELGMLTTLFNRMT 454 346 A_group_6
PLSELAAGTRAVA ELGMLTTLFNRMT 454 346 A_group_6
PLLILAEGTQAVA ELGVLTQSFNRMT 456 347 A_group_6
PLLILAEGTQAVA ELGVLTQSFNRMT 456 347 A_group_6
PLRTLARGTRAVA ELGVLIQSFNRMT 457 348 A_group_6
PLRTLARGTRAVA ELGVLIQSFNRMT 457 348 A_group_6
PLLRLAAGTQAVG EVGQLTRSFNAMT 459 349 A_group_6
PLLRLAAGTQAVG EVGQLTRSFNAMT 459 349 A_group_6
PLLMLLKGTQAVA ELGMLTRQFNVMT 462 350 A_group_6
PLLMLLKGTQAVA ELGMLTRQFNVMT 462 350 A_group_6
PLLLLAEGTKAVA ELGTLTQSFNMMT 466 351 A_group_6
PLLLLAEGVKQVA ELGGLTRSFADMT 467 352 A_group_6
PVNDLVAAARRVA EIGTLGSAFNRMT 472 353 A_group_63
PLLDLFYATRKVQ EMSTLARAFNQMT 473 354 A_group_63
PIGRLASAAHTVR ELHALGHSFNAMT 474 355 A_group_63
PLAELVSAARTIG EIGLLARAFNRMT 475 356 A_group_63
PLTDLVAAARKVG EIGLLNRAFNRMT 476 357 A_group_63
PLTDLVAAARKVG EIGLLNRAFNRMT 476 357 A_group_63
PISHMVEAVRRIG EIGTLGFAFNRMA 477 358 A_group_63
PISHMVEAVRRIG EIGTLGFAFNRMA 477 358 A_group_63
PIRTLISAADEVS DLANLGETFNKMT 479 359 A_group_63
PIRLLISAADDVA DIGQLSKTFNY

PIVDAAGAVSKIG ELATLGANINDMA 619 449 A_mgroup_100
PILEASSAVQKLA ELASLGSSINRMV 620 450 A_mgroup_100
PILEASSAVQKLA ELASLGSSINRMV 620 450 A_mgroup_100
PITDSADAVDAIG ELAQLGGNINQMA 621 451 A_mgroup_100
PITDSADAVDAIG ELAQLGGNINQMA 621 451 A_mgroup_100
PIQRLATAAEQVR EIGDLSRSLKAMT 622 452 A_mgroup_101
PIRRLALAAERVR EIGELSLALKEMT 623 453 A_mgroup_101
PIRRLALAAERVR EIGELSLALKEMT 623 453 A_mgroup_101
PLRKLSAAADRVR EVGHLSTSIREMT 624 454 A_mgroup_101
PLRKLSAAADRVR EVGHLSTSIREMT 624 454 A_mgroup_101
PIRHLAEAAERVR EIGHLSGALRDMT 625 455 A_mgroup_101
PIRHLAEAAERVR EIGHLSGALRDMT 625 455 A_mgroup_101
PLLRLAASAHVIR EIGEVARALQDSA 627 456 A_mgroup_101
PLLRLAASAHVIR EIGEVARALQDSA 627 456 A_mgroup_101
PVRRLAAAADRVR EIGHLARSFREMT 628 457 A_mgroup_101
PVRRLAAAADRVR EIGHLARSFREMT 628 457 A_mgroup_101
PLLRLAGAADRVR ELGDLSAALREMT 630 458 A_mgroup_101
PLLRLAGAADRVR ELGDLSAALREMT 630 458 A_mgroup_101
PVLRMASAADRVR ELGDLSRALEEMT 631 459 A_mgroup_101
PVRRLAQAAETVR EIGELSGALREMT 632 460 A_mgroup_101
PLRRLARAAVRVR EIGTLA

...

NLALLKSASQRIL EIGKTMLAFNQMV 6098 4990 BACKGROUND
PLMSMSDHFMKLA EIGEVFEQLKLMQ 6099 4991 BACKGROUND
PLMSMSDHFMKLA EIGEVFEQLKLMQ 6099 4991 BACKGROUND
IVNQIKRVVERLA ELCALGADLDRVI 6100 4992 BACKGROUND
RMDELVTLARQVR EIGILSQAFSDVV 6101 4993 BACKGROUND
RMDELVTLARQVR EIGILSQAFSDVV 6101 4993 BACKGROUND
RLKRIGEVANAIS FIGDMANSFNLMS 6102 4994 BACKGROUND
RLKRIGEVANAIS FIGDMANSFNLMS 6102 4994 BACKGROUND
RVGRLKASAVEIS EIDDLTEAIRTMQ 6103 4995 BACKGROUND
RVGRLKASAVEIS EIDDLTEAIRTMQ 6103 4995 BACKGROUND
PVRRMLVNFEHLA ELGRMQKAFARML 6104 4996 BACKGROUND
PVRRMLVNFEHLA ELGRMQKAFARML 6104 4996 BACKGROUND
PLVALADTAQRIS EVTNLAQALESMQ 6105 4997 BACKGROUND
PLVALADTAQRIS EVTNLAQALESMQ 6105 4997 BACKGROUND
PVQYWVSRIQAIA EIGILGNNIQQMQ 6106 4998 BACKGROUND
PFTQLVEAMKHND EIGALFSGYNEMI 6107 4999 BACKGROUND
PFTQLVEAMKHND EIGALFSGYNEMI 6107 4999 BACKGROUND
SFEEMSNAAFRIS SLGEVGHAFNSMV 6108 5000 BACKGROUND
SFEEMSNAAFRIS SLGEVGHAFNSMV 6108 5000 BACKGROUND
YIRKIMVITSDIA ELGTLAKNVNNMK 6109 5001 BACKGROUND
PINAIVKDAARVA EIDNLQ

PLRSLTAAAEQVA DIDRLAQAFRHMQ 6203 5084 BACKGROUND
PIMDIAEISQQLD ETTQLISAFKSMS 6204 5085 BACKGROUND
PIMDIAEISQQLD ETTQLISAFKSMS 6204 5085 BACKGROUND
PITGIAASVSRVD EIVILASGFNQMV 6205 5086 BACKGROUND
SIENVAAKMNLFD ELRIIESSFNRML 6206 5087 BACKGROUND
SIENVAAKMNLFD ELRIIESSFNRML 6206 5087 BACKGROUND
PLRQSLDAAMAIA ELLELNLALGQMV 6207 5088 BACKGROUND
PLRQSLDAAMAIA ELLELNLALGQMV 6207 5088 BACKGROUND
PLASVTKSIEDLT ETELITDAFNKML 6208 5089 BACKGROUND
PLASVTKSIEDLT ETELITDAFNKML 6208 5089 BACKGROUND
PLNTLMQLTDNIA EVANLAHSIELME 6209 5090 BACKGROUND
PLNTLMQLTDNIA EVANLAHSIELME 6209 5090 BACKGROUND
HVAVISETLDQVT DLARIGAKVDQMA 6210 5091 BACKGROUND
HVAVISETLDQVT DLARIGAKVDQMA 6210 5091 BACKGROUND
PLRRVAQFASELT EIDLVVEGFSVLQ 6211 5092 BACKGROUND
RLTLGAQAARKIA EVAAFASAVDTAV 6212 5093 BACKGROUND
RLTLGAQAARKIA EVAAFASAVDTAV 6212 5093 BACKGROUND
IIDQINQVMRKVN RLTELKTFINQTV 6213 5094 BACKGROUND
IIDQINQVMRKVN RLTELKTFINQTV 6213 5094 BACKGROUND
NLKRLKRGTQRLG ELGDLAIAFNTMS 6214 5095 BACKGROUND
NLKRLKRGTQRLG ELGDLA

SIESATNVATELS RSDKLGHAMNSLA 6312 5180 BACKGROUND
NIDIITDGLSTLA EMGQISQSVNNLA 6313 5181 BACKGROUND
PLEWAARKAEAMS EVAALVKALNGML 6314 5182 BACKGROUND
PIEALAKEVRELE ELTSLVRNLNRLL 6315 5183 BACKGROUND
HLNKISQYAHTLT ELDTVVTALNEMQ 6316 5184 BACKGROUND
HLNKISQYAHTLT ELDTVVTALNEMQ 6316 5184 BACKGROUND
PLQQLSEQMQNQA ELQVLIDSFGAMR 6318 5185 BACKGROUND
PIKYLYSTIEKQK EIGVTIKEFFNLQ 6319 5186 BACKGROUND
PIKYLYSTIEKQK EIGVTIKEFFNLQ 6319 5186 BACKGROUND
YLNRIMKDTKDMA PLASHAENLNHLR 6320 5187 BACKGROUND
YLNRIMKDTKDMA PLASHAENLNHLR 6320 5187 BACKGROUND
PLKALTRRVGDFA DIQGLIDAHNAME 6321 5188 BACKGROUND
KLESLMFAAKNVI IFARLAHTFNNMR 6322 5189 BACKGROUND
KLESLMFAAKNVI IFARLAHTFNNMR 6322 5189 BACKGROUND
RIERLAQRAEQVG ALGRLARQLTHSH 6323 5190 BACKGROUND
RIERLAQRAEQVG ALGRLARQLTHSH 6323 5190 BACKGROUND
QIAEITDALIDVK LVAPLAYEINEIV 6324 5191 BACKGROUND
RFESLAEHAQLVT EFNALADSLQQMS 6325 5192 BACKGROUND
NIHRLVAYARSLS DLTQLAQAMDEMR 6326 5193 BACKGROUND
NIHRLVAYARSLS DLTQLAQAMDEMR 6326 5193 BACKGROUND
PIVLIAKKVSESR EIRHLI

PIQRLAKDVSKIE EVHSLSRTIQAML 6415 5278 BACKGROUND
PIQRLAKDVSKIE EVHSLSRTIQAML 6415 5278 BACKGROUND
SIEALRHVVHLQR DVRGLAADFNHLT 6416 5279 BACKGROUND
SIEALRHVVHLQR DVRGLAADFNHLT 6416 5279 BACKGROUND
SLETLAEAARRLS EVRELATATAAMA 6417 5280 BACKGROUND
SLETLAEAARRLS EVRELATATAAMA 6417 5280 BACKGROUND
PLRRLAAIAAHVA EVGIVGHALNRLL 6418 5281 BACKGROUND
PLRRLAAIAAHVA EVGIVGHALNRLL 6418 5281 BACKGROUND
PINRLVRRAEEYR EFRQLSGALNRML 6419 5282 BACKGROUND
PINRLVRRAEEYR EFRQLSGALNRML 6419 5282 BACKGROUND
PLNMVGEALQGVL EVRPLLRYIDKLM 6420 5283 BACKGROUND
PLNMVGEALQGVL EVRPLLRYIDKLM 6420 5283 BACKGROUND
KLKLFSQVSEKVA EYDQALDAMEHMR 6422 5284 BACKGROUND
KLKLFSQVSEKVA EYDQALDAMEHMR 6422 5284 BACKGROUND
RLQLLVTAVRRYR EVDLLGTAISDMS 6423 5285 BACKGROUND
RLQLLVTAVRRYR EVDLLGTAISDMS 6423 5285 BACKGROUND
PLKQLADLVNGVA EIGILAQTLETAM 6424 5286 BACKGROUND
PLKQLADLVNGVA EIGILAQTLETAM 6424 5286 BACKGROUND
PIKRLTRYANKIR ELRDMGLALEDMR 6426 5287 BACKGROUND
PIKRLTRYANKIR ELRDMGLALEDMR 6426 5287 BACKGROUND
NMKKVYQITKNFS VLYGLY

5314

In [20]:
data_df.to_pickle(os.path.join(data_dir, 'af2_newrun.p'))

In [21]:
# checked, OK